# A Walkthrough: Analyzing a Cell with patchAgent

*A step-by-step demo of an interactive analysis session.*

---

In this post, we'll walk through a complete patchAgent session — from launching the CLI to extracting spike features, passive properties, and quality metrics from a real patch-clamp recording. Screenshots show the actual terminal output at each step.

> **Note:** patchAgent runs as a terminal application. The cells below are narrative + screenshots, not live code. If you want to follow along, install patchAgent and use your own data file.

## 1. Installation

patchAgent is installed from source with the `[cli]` extras for the interactive chat interface:

```bash
git clone https://github.com/smestern/patchAgent.git
cd patchAgent
pip install -e ".[cli]"
```

This installs the core analysis libraries (pyABF, IPFX, NumPy, SciPy) plus the CLI dependencies (Typer, prompt-toolkit, Rich).

## 2. Launching the Agent

Start an interactive session from your terminal:

```bash
patchagent chat
```

You can also pre-load a file and specify an output directory:

```bash
patchagent chat --file data/sample_abfs/cell_001.abf --output-dir ./results
```

<!-- SCREENSHOT: Terminal showing the patchAgent startup banner and prompt -->
![patchAgent startup](assets/01-startup.png)
*The agent starts up, displays its banner, and presents the `🧪 You ❯` prompt. It's ready to accept natural-language commands.*

## 3. Loading a File

Use the `/load` slash command (or just ask the agent to load a file in plain English):

```
🧪 You ❯ /load data/sample_abfs/cell_001.abf
```

The agent calls its `load_file` tool, reads the ABF, and reports what it found:

<!-- SCREENSHOT: Terminal output after loading a file — sweep count, sampling rate, protocol, clamp mode -->
![File loaded](assets/02-load-file.png)
*The agent reports: file type, number of sweeps, sampling rate, protocol name, and clamp mode. This metadata tells you immediately what kind of experiment this is.*

You can also inspect the sweep list:

```
🧪 You ❯ /sweeps
```

<!-- SCREENSHOT: Sweep list showing sweep numbers and stimulus amplitudes -->
![Sweep list](assets/03-sweep-list.png)
*Each sweep is listed with its stimulus amplitude (in pA). The range of current steps tells you this is a standard current-step protocol — exactly what we need for spike and passive property analysis.*

## 4. Passive Properties

Let's start with the basics — passive membrane properties from subthreshold (non-spiking) sweeps.

```
🧪 You ❯ What is the input resistance of this cell?
```

Behind the scenes, the agent:
1. Identifies sweeps with hyperpolarizing current steps
2. Measures the steady-state voltage deflection in each
3. Fits V vs. I (Ohm's law) to calculate Rm
4. Reports the result with uncertainty

<!-- SCREENSHOT: Agent response showing input resistance calculation with value, uncertainty, N, and any flags -->
![Input resistance](assets/04-input-resistance.png)
*The agent reports Rm with units (MΩ), uncertainty (± SEM), and the number of sweeps used. If the value were outside the typical physiological range (10–2000 MΩ), it would flag a warning.*

We can ask for more passive properties:

```
🧪 You ❯ Calculate the membrane time constant and sag ratio too
```

<!-- SCREENSHOT: Agent response showing tau (with exponential fit details) and sag ratio -->
![Tau and sag](assets/05-tau-sag.png)
*The time constant (τ) comes from an exponential fit to the voltage response. The sag ratio quantifies how much the voltage "sags" back from its peak deflection during a hyperpolarizing step — a hallmark of the Ih current.*

## 5. Spike Detection and Features

Now the fun part — action potentials.

```
🧪 You ❯ Detect spikes in sweep 10 and show me the features
```

The agent uses its `detect_spikes` tool (which wraps IPFX's dV/dt-based detector) and then calls `extract_spike_features` for detailed per-spike metrics.

<!-- SCREENSHOT: Agent response showing spike count, and a table of per-spike features (threshold, amplitude, half-width, etc.) -->
![Spike detection](assets/06-spike-detection.png)
*The agent found action potentials using a 20 mV/ms dV/dt threshold (the default, matching IPFX conventions). Each spike's threshold voltage, peak amplitude, and half-width are reported.*

We can also ask for train-level features:

```
🧪 You ❯ What about spike train features — is this cell adapting?
```

<!-- SCREENSHOT: Spike train analysis showing adaptation index, ISI CV, burst detection, etc. -->
![Spike train features](assets/07-spike-train.png)
*Train-level analysis reveals adaptation ratio, interspike interval (ISI) coefficient of variation, latency to first spike, and burst detection. These features help classify the cell's firing pattern.*

## 6. Building an f-I Curve

An f-I (frequency–current) curve shows how a neuron's firing rate increases with injected current — one of the most common analyses in electrophysiology.

```
🧪 You ❯ Generate an f-I curve for this cell
```

The agent:
1. Detects spikes across all suprathreshold sweeps
2. Maps stimulus amplitude → firing rate for each sweep
3. Fits the resulting curve (linear or square-root model)
4. Generates a plot and saves it to the output directory

<!-- SCREENSHOT: f-I curve plot generated by the agent — scatter points + fit line, axis labels, title -->
![f-I curve](assets/08-fi-curve.png)
*The f-I curve shows firing frequency (Hz) vs. injected current (pA). The agent fits a model to the data and reports the slope (gain) and rheobase (minimum current for spiking). The figure is automatically saved to the output directory.*

## 7. Quality Control

Before trusting any of these numbers, we should check data quality.

```
🧪 You ❯ Run quality control checks on this recording
```

The agent runs its QC suite:
- **Baseline stability**: Is the resting membrane potential drifting over time?
- **Noise levels**: Is the RMS noise acceptable for this preparation?
- **Signal integrity**: Any clipping or saturation in the voltage traces?

<!-- SCREENSHOT: QC report showing baseline stability metrics, noise measurement, and overall pass/flag assessment -->
![Quality control](assets/09-qc-report.png)
*The QC report checks each criterion and flags potential issues. A clean recording passes all checks; a problematic one gets explicit warnings about which sweeps or metrics are concerning.*

This is one of patchAgent's most valuable features — it applies the same quality standards a senior electrophysiologist would, but consistently and automatically.

## 8. Asking Follow-Up Questions

The agent maintains conversational context, so you can ask follow-up questions naturally:

```
🧪 You ❯ Which sweep had the most spikes?

🧪 You ❯ Plot the voltage trace for that sweep

🧪 You ❯ Is the spike amplitude decreasing over the train? That might indicate depolarization block.
```

<!-- SCREENSHOT: A conversational exchange where the agent answers follow-ups using context from earlier in the session -->
![Follow-up questions](assets/10-followup.png)
*The agent remembers what file is loaded, what analyses have been run, and can reference earlier results. You don't have to re-specify everything — just ask your question.*

## 9. Saving Your Work

When you're done, save the entire conversation as a Markdown file:

```
🧪 You ❯ /save analysis_cell001.md
```

This creates a readable record of every question you asked, every analysis the agent ran, and every result it reported — a complete audit trail for your lab notebook.

Your output directory also contains:
- All generated figures (PNG)
- All executed scripts (Python) — timestamped and hash-named for reproducibility
- Any exported data (CSV)

<!-- SCREENSHOT: The output directory listing showing saved figures, scripts, and the conversation markdown -->
![Output directory](assets/11-output-dir.png)
*Everything the agent produced is saved to a single output directory, making it easy to include in a paper, share with collaborators, or re-run later.*

## What Just Happened?

In about 10 minutes of conversation, we:

| Step | What We Did | Traditional Approach |
|------|-------------|---------------------|
| Load data | One `/load` command | 10–20 lines of pyABF code |
| Input resistance | One English question | Write V/I fitting code, select sweeps, handle edge cases |
| Spike detection | One English question | Configure IPFX, set thresholds, parse output |
| f-I curve | One English question | Loop over sweeps, count spikes, fit model, generate plot |
| Quality control | One English question | Write custom QC checks from scratch |
| Save everything | One `/save` command | Manual file management |

The agent handled the boilerplate — loading files, calling the right analysis functions with the right parameters, validating results, and saving outputs. The scientist focused on asking the right *questions*, which is where their expertise matters most.

And crucially, every step used validated, peer-reviewed methods (IPFX for spike detection, Ohm's law for Rm, exponential fitting for τ) — not ad-hoc code that might contain subtle bugs.

---

*Previous: [← Why We Built patchAgent](02-why-patch-agent.md)*  
*Next: [Guardrails: Keeping AI Honest in Science →](04-guardrails.md)*